In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from dataset.mnist import load_mnist

## 신경망 학습

### 손실함수란?
    1. 오차제곱합 SSE(sum of square for error) 
    2. 교차 엔트로피 오차  CEE(cross entropy error)

### 1. 오차제곱합 SSE

In [5]:
# y => 신경망의 출력(신경망이 추정한 값)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

In [6]:
len(y)

10

In [7]:
# 정답 레이블 
t = [0,0,1,0,0,0,0,0,0,0]

In [8]:
len(t)

10

In [9]:
# sum square error function 
def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [11]:
# 오차제곱합 구하기
sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

#### 위에서 나온 오차제곲합 => 0.0975
    => 신경망 출력 y값이 '2'일 확률이 가장 높다고 추정한 y값을 넣었을때 (0.6)

### '7'일 확률이 가장 높다고 추정한값을 넣을때 오차제곱합을 알아보자 

In [12]:
y_new = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

In [13]:
len(y_new)

10

In [14]:
# 오차 제곱합 구하기 
sum_squares_error(np.array(y_new), np.array(t))

0.5975

#### '7'일 확률이 가장 높다고 추정한 신경망 출력값 넣었을때는 
    => 오차 제곱합이 0.5975로 실제로 정답을 맞춘 위의 오차제곱합 0.0975보다 높다

### 2. 교차 엔트로피 오차 CEE(cross entropy error)
    => 로그함수를 이용한다

In [15]:
# 교차 엔트로피 오차 
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

### t 정답 레이블 => 정답은 2


#### 1. 정답일때의 출력 0.6 일때의 신경망 출력값일때 

In [16]:
# 교차 엔트로피 오차 구하기 
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

#### 2. 정답이 2이지만, 정답을 '7'로 설정한 0.6출력일떄의 신경망일때

In [17]:
cross_entropy_error(np.array(y_new), np.array(t))

2.302584092994546

#### 0.5108 < 2.3025 로 교차 엔트로피 오차도 높아졌다 오답일때 

#### 미니배치 학습이란?
    => 훈련데이터를 통해 학습을 
    => 훈련데이터에 대한 '손실 함수' 값을 구함으로써 평가한다 
    => 손실함수가 '최소화' 되는 지점이 정답일 확률이 높고 
    => 손실함수를 최대한 줄여주는 매개변수를 찾아낸다.

#### 데이터가 N개 라면 Tnk는 N번째의 K번째 값 
    => 허나 마지막에 N개의 데이터로 나눠서 '평균 손실 함수' 를 구한다
    => 미니배치 학습이란 훈련데이터의 '일부'를 추려 학습하는걸 
    => 미니 배치 학습이라고 한다 

In [18]:
#load data 
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True,
                                                 one_hot_label=True)


In [19]:
x_train.shape

(60000, 784)

In [20]:
t_train.shape

(60000, 10)

In [23]:

x_train[0].shape

(784,)

In [26]:
# 훈련데이터 수 
train_size = x_train.shape[0]
train_size

60000

In [27]:
# 미니배치 사이즈 => 10
batch_size=10

In [31]:
# 훈련데이터에서 무작위로 배치 사이즈만큼 10장 추출 
batch_mask = np.random.choice(train_size, batch_size)

In [32]:
#위의 값 확인 
batch_mask

array([42871, 41829, 26260, 24660, 18803, 49438, 30294,  2559,  8996,
       50617])

In [34]:
# 무작위로 추출해낸 10개 배열을 인덱스로 삼아서 훈련데이터 추출 
x_batch = x_train[batch_mask]
x_batch.shape

(10, 784)

In [36]:
# 무작위로 추출해낸 10개의 데이터를 인덱스로 삼아 그 해당 정답 데이터 
t_batch = t_train[batch_mask]
t_batch.shape

(10, 10)

## 그럼 미니배치같은 배치데이터 지원하는
    => 교차 엔트로피 오차는 어떻게 구현할까요??

In [40]:
x_batch.size

7840

### 정답 레이블이 one-hot-encoding을 통해 0 or 1일 경우

In [42]:
def cross_entropy_error(y, t):
    if y.ndim == 1: # y(신경망 배열) 값이 1차원이라면 
        # 1차원으로 바꿔준다
        t = t.reshape(1, t.size) # np.size => 전체 원소 개수 반환
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size # 평균값으로 나눠줌

### 정답 레이블이 one-hot-encoding을 통하지 않아 2 or 7일경우

In [43]:
def cross_entropy_error(y, t):
    if y.ndim == 1: # y 신경망 출력값이 1차원일 경우 
        # 1차우너으로 바꿔준다 
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
        

## 왜 손실함수를 설정하나요?
    => 기울기로만 하면 0이 되는 지점이 많이 발생해서 연속적이지 않다 
    => 우리는 최적의 매개변수(가중치와 편향)을 탐색하기 위해서 
    => 손실함수의 값이 최소화 하는 매개변수 값을 찾는다 
    => 이떄 매개변수의 미분(기울기)를 계산하고, 그 미분값을 단서로 매개변수 값을 
    서서히 갱신하는 과정 반복
    => 손실함수에 대한 미분=> 가중치 매개변수의 값을 조금씩 변화할때 
    손실함수가 어떻게 변하나? 커지냐 작아지냐? 를 통해서 
    => 미분값이 음수일때는 가중치 매개변수를 양의 방향으로 
    or 미분값이 양수일때는 가중치 매개변수를 음의 방향으로